# ResNet + FPN → DETR


Постепенное построение DETR архитектуры через наследование, начиная с базовой ResNet + FPN. Каждый шаг добавляет один ключевой компонент, приближая нас к полноценной реализации.

**Основная идея DETR:** вместо anchor-based детекции используем набор learnable queries, каждый из которых ищет один объект через cross-attention к feature map. Это упрощает pipeline и делает детекцию end-to-end differentiable.

**Ключевые отличия от ResNet + FPN:**
- ResNet + FPN: $f: \mathbb{R}^{H \times W \times 3} \to \mathbb{R}^{H' \times W' \times C}$ — плотная feature map для anchor-based детекции
- DETR: $f: \mathbb{R}^{H \times W \times 3} \to \{(c_i, b_i)\}_{i=1}^N$ — набор из $N$ пар (класс, bbox)

Это позволяет избежать NMS и anchor tuning.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.models import resnet50
import math


### Зачем начинать с ResNet + FPN?

ResNet — это мощный backbone для извлечения признаков из изображений. Он использует residual connections, которые позволяют обучать очень глубокие сети.

FPN (Feature Pyramid Network) добавляет multi-scale feature extraction через top-down pathway с lateral connections. Это позволяет комбинировать высокоуровневую семантическую информацию с низкоуровневыми деталями.

В классической object detection ResNet + FPN используется как backbone для генерации dense predictions с anchors. Но в DETR мы будем использовать только feature extraction часть, а detection head заменим на transformer с queries.

Архитектура ResNet + FPN:
$$
\text{Image} \xrightarrow{\text{ResNet}} \{C_2, C_3, C_4, C_5\} \xrightarrow{\text{FPN}} \{P_2, P_3, P_4, P_5\}
$$

где $C_i$ — выходы ResNet слоев с stride $2^i$, а $P_i$ — FPN features той же размерности.


## Step 0: Base ResNet + FPN


In [2]:
class FPN(nn.Module):
    def __init__(self, in_channels_list, out_channels=256):
        super().__init__()
        self.lateral_convs = nn.ModuleList([
            nn.Conv2d(in_channels, out_channels, 1) for in_channels in in_channels_list
        ])
        self.fpn_convs = nn.ModuleList([
            nn.Conv2d(out_channels, out_channels, 3, padding=1) for _ in in_channels_list
        ])
    
    def forward(self, features):
        laterals = [lateral_conv(features[i]) for i, lateral_conv in enumerate(self.lateral_convs)]
        
        for i in range(len(laterals) - 1, 0, -1):
            laterals[i - 1] += F.interpolate(laterals[i], size=laterals[i - 1].shape[-2:], mode='nearest')
        
        fpn_features = [fpn_conv(lateral) for fpn_conv, lateral in zip(self.fpn_convs, laterals)]
        return fpn_features

class ResNetFPN(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        backbone = resnet50(pretrained=pretrained)
        self.conv1 = backbone.conv1
        self.bn1 = backbone.bn1
        self.relu = backbone.relu
        self.maxpool = backbone.maxpool
        
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
        
        self.fpn = FPN([256, 512, 1024, 2048], out_channels=256)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        c1 = self.layer1(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)
        
        features = self.fpn([c1, c2, c3, c4])
        return features

class Step0(ResNetFPN):
    pass

model = Step0(pretrained=False)
features = model(torch.randn(2, 3, 640, 640))
print(f"Step 0: FPN outputs with shapes {[f.shape for f in features]}")


/home/tam2511/venvs/train_py10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tam2511/venvs/train_py10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Step 0: FPN outputs with shapes [torch.Size([2, 256, 160, 160]), torch.Size([2, 256, 80, 80]), torch.Size([2, 256, 40, 40]), torch.Size([2, 256, 20, 20])]


### Зачем нужна единая feature map?

В DETR мы не используем multi-scale features напрямую, как в классических детекторах. Вместо этого берём feature map с одного уровня и работаем с ней через transformer.

Обычно используют выход последнего слоя FPN (или даже напрямую выход ResNet без FPN). Для простоты возьмём последний уровень FPN: $P_5 \in \mathbb{R}^{B \times 256 \times H' \times W'}$, где $H' = H/32$, $W' = W/32$.

Эта feature map будет использоваться как "память" для transformer decoder — queries будут смотреть на неё через cross-attention, чтобы найти объекты.

В более продвинутых версиях DETR (Deformable DETR) используются multi-scale features с deformable attention, но для базовой версии достаточно одного уровня.


## Step 1: Extract Single Feature Map


In [3]:
class Step1(ResNetFPN):
    def __init__(self, emb_dim=256):
        super().__init__(pretrained=False)
        self.emb_dim = emb_dim
    
    def forward(self, x):
        features = super().forward(x)
        return features[-1]

model = Step1(emb_dim=256)
feature_map = model(torch.randn(2, 3, 640, 640))
print(f"Step 1: Single feature map {feature_map.shape}")


Step 1: Single feature map torch.Size([2, 256, 20, 20])


### Переход к query-based подходу

Ключевая идея DETR: вместо плотных predictions на каждом пикселе или для каждого anchor, мы создаём фиксированное число learnable queries $Q \in \mathbb{R}^{N \times d}$, где $N=100$ — максимальное количество объектов, которое мы хотим детектировать на одном изображении.

Каждый query — это learned embedding, который будет "искать" один объект. На выходе каждый query предсказывает:
- Класс объекта: $p_i \in \mathbb{R}^{C+1}$ (дополнительный класс "no object")
- Bounding box: $b_i \in \mathbb{R}^4$ в формате $(c_x, c_y, w, h)$ normalized

Пока просто пропускаем queries через линейные слои:
$$
\text{class\_logits} = \text{Linear}(Q) \in \mathbb{R}^{B \times N \times (C+1)}
$$
$$
\text{bbox\_pred} = \text{Sigmoid}(\text{Linear}(Q)) \in \mathbb{R}^{B \times N \times 4}
$$

Queries пока статичные, но скоро добавим transformer для их обогащения информацией из изображения.


## Step 2: Add Queries


In [4]:
class Step2(Step1):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100):
        super().__init__(emb_dim)
        self.num_classes = num_classes
        self.num_queries = num_queries
        self.queries = nn.Embedding(num_queries, emb_dim)
        self.class_head = nn.Linear(emb_dim, num_classes + 1)
        self.bbox_head = nn.Linear(emb_dim, 4)
    
    def forward(self, x):
        B = x.shape[0]
        features = super().forward(x)
        
        queries = self.queries.weight.unsqueeze(0).expand(B, -1, -1)
        class_logits = self.class_head(queries)
        bbox_pred = self.bbox_head(queries).sigmoid()
        
        return {'class_logits': class_logits, 'bbox_pred': bbox_pred, 'features': features}

model = Step2(num_classes=80)
out = model(torch.randn(2, 3, 640, 640))
print(f"Step 2: class_logits={out['class_logits'].shape}, bbox_pred={out['bbox_pred'].shape}")


Step 2: class_logits=torch.Size([2, 100, 81]), bbox_pred=torch.Size([2, 100, 4])


### Почему transformer нужны позиционные эмбеддинги?

Transformer изначально придуман для последовательностей, где нет пространственной структуры. Attention механизм сам по себе permutation-invariant — он не знает, где находится каждый элемент.

Для изображений это проблема: пиксель в левом верхнем углу должен иметь другое представление, чем пиксель справа внизу.

Решение — добавить 2D sinusoidal positional encoding:
$$
\text{PE}(x, y, 2i) = \sin\left(\frac{x}{10000^{2i/d}}\right), \quad \text{PE}(x, y, 2i+1) = \cos\left(\frac{x}{10000^{2i/d}}\right)
$$

Делаем это отдельно для координат $x$ и $y$, потом конкатенируем. Получаем $\text{pos\_emb} \in \mathbb{R}^{B \times d \times H' \times W'}$.

Также создаём learnable позиционные embeddings для queries — каждый query должен знать свою позицию в наборе.


## Step 3: Add Positional Encoding


In [5]:
class PositionEmbedding2D(nn.Module):
    def __init__(self, dim=256, temperature=10000):
        super().__init__()
        self.dim = dim
        self.temperature = temperature
    
    def forward(self, x):
        B, C, H, W = x.shape
        y_embed = torch.arange(H, dtype=torch.float32, device=x.device).view(H, 1).repeat(1, W)
        x_embed = torch.arange(W, dtype=torch.float32, device=x.device).view(1, W).repeat(H, 1)
        
        dim_t = torch.arange(self.dim // 4, dtype=torch.float32, device=x.device)
        dim_t = self.temperature ** (2 * dim_t / (self.dim // 4))
        
        pos_x = x_embed[:, :, None] / dim_t
        pos_y = y_embed[:, :, None] / dim_t
        
        pos_x = torch.cat([pos_x.sin(), pos_x.cos()], dim=-1)
        pos_y = torch.cat([pos_y.sin(), pos_y.cos()], dim=-1)
        
        pos = torch.cat([pos_y, pos_x], dim=-1).permute(2, 0, 1).unsqueeze(0).expand(B, -1, -1, -1)
        return pos

class Step3(Step2):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100):
        super().__init__(num_classes, emb_dim, num_queries)
        self.pos_emb = PositionEmbedding2D(emb_dim)
        self.query_pos = nn.Embedding(num_queries, emb_dim)
    
    def forward(self, x):
        out = super().forward(x)
        features = out['features']
        pos_emb = self.pos_emb(features)
        
        out['pos_emb'] = pos_emb
        out['query_pos'] = self.query_pos.weight
        return out

model = Step3(num_classes=80)
out = model(torch.randn(2, 3, 640, 640))
print(f"Step 3: positional encoding added, pos_emb={out['pos_emb'].shape}")


Step 3: positional encoding added, pos_emb=torch.Size([2, 256, 20, 20])


### Зачем нужен Transformer Encoder?

В оригинальном DETR перед тем, как queries начнут смотреть на feature map через decoder, сама feature map обогащается через Transformer Encoder.

Encoder применяет self-attention к feature map, позволяя каждой позиции "увидеть" всё изображение и обновить свои признаки с учётом глобального контекста. Это критично для качества детекции.

Архитектура:
1. Feature map flatten: $F \in \mathbb{R}^{B \times C \times H \times W} \to \mathbb{R}^{B \times (H \cdot W) \times C}$
2. Добавляем позиционные embeddings: $F + \text{pos\_emb}$
3. Пропускаем через Transformer Encoder:
   $$
   F_{\text{enc}} = \text{TransformerEncoder}(F + \text{pos\_emb})
   $$

Внутри encoder layer происходит:
- Self-attention между всеми позициями feature map
- Feed-forward network

После encoder получаем обогащённую feature map $F_{\text{enc}}$, которая будет использоваться как memory для decoder.


## Step 4: Add Transformer Encoder


In [6]:
class Step4(Step3):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100, nhead=8, enc_layers=6):
        super().__init__(num_classes, emb_dim, num_queries)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim,
            nhead=nhead,
            dim_feedforward=emb_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=enc_layers)
    
    def forward(self, x):
        B = x.shape[0]
        features = Step1.forward(self, x)
        pos_emb = self.pos_emb(features)
        
        mem = features.flatten(2).permute(0, 2, 1)
        mem_pos = pos_emb.flatten(2).permute(0, 2, 1)
        
        mem = self.transformer_encoder(mem + mem_pos)
        
        queries = self.queries.weight.unsqueeze(0).expand(B, -1, -1)
        class_logits = self.class_head(queries)
        bbox_pred = self.bbox_head(queries).sigmoid()
        
        return {'class_logits': class_logits, 'bbox_pred': bbox_pred, 'mem': mem, 'mem_pos': mem_pos}

model = Step4(num_classes=80, enc_layers=6)
out = model(torch.randn(2, 3, 640, 640))
print(f"Step 4: encoder applied, mem={out['mem'].shape}, class_logits={out['class_logits'].shape}")


Step 4: encoder applied, mem=torch.Size([2, 400, 256]), class_logits=torch.Size([2, 100, 81])


### Добавляем Transformer Decoder

Теперь queries могут смотреть на обогащённую encoder'ом feature map через cross-attention. Используем стандартный TransformerDecoder из PyTorch.

Как это работает:
1. Берём encoded memory $\text{mem}$ из предыдущего шага
2. Queries получают свои позиции: $Q + Q_{\text{pos}}$
3. Пропускаем через Transformer Decoder:
   $$
   Q' = \text{TransformerDecoder}(Q + Q_{\text{pos}}, \text{mem} + \text{mem\_pos})
   $$

Внутри каждого decoder layer происходит:
- Self-attention между queries — queries обмениваются информацией друг с другом
- Cross-attention queries → memory — queries смотрят на encoded feature map и ищут объекты
- Feed-forward network — нелинейная трансформация

После этого queries становятся content-aware — каждый query научился фокусироваться на своём объекте.


## Step 5: Add Transformer Decoder


In [7]:
class Step5(Step4):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100, nhead=8, enc_layers=6, dec_layers=6):
        super().__init__(num_classes, emb_dim, num_queries, nhead, enc_layers)
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=emb_dim,
            nhead=nhead,
            dim_feedforward=emb_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=dec_layers)
    
    def forward(self, x):
        B = x.shape[0]
        out = super().forward(x)
        
        mem = out['mem']
        mem_pos = out['mem_pos']
        
        queries = self.queries.weight.unsqueeze(0).expand(B, -1, -1)
        q_pos = self.query_pos.weight.unsqueeze(0).expand(B, -1, -1)
        
        queries = self.transformer_decoder(queries + q_pos, mem + mem_pos)
        
        class_logits = self.class_head(queries)
        bbox_pred = self.bbox_head(queries).sigmoid()
        
        return {'class_logits': class_logits, 'bbox_pred': bbox_pred, 'mem': mem, 'mem_pos': mem_pos}

model = Step5(num_classes=80, enc_layers=6, dec_layers=6)
out = model(torch.randn(2, 3, 640, 640))
print(f"Step 5: decoder applied, class_logits={out['class_logits'].shape}, bbox_pred={out['bbox_pred'].shape}")


Step 5: decoder applied, class_logits=torch.Size([2, 100, 81]), bbox_pred=torch.Size([2, 100, 4])


### Deep Supervision для ускорения обучения

Deep supervision — это техника, когда мы считаем loss не только на финальном выходе, но и на промежуточных слоях transformer. Зачем?

1. Помогает градиентам лучше проходить через глубокую сеть
2. Заставляет ранние слои transformer сразу учиться предсказывать разумные bbox и классы
3. Даёт регуляризацию — модель не может полагаться только на последний слой

Модифицируем TransformerDecoder, чтобы он возвращал выходы всех промежуточных слоёв:
$$
[Q^{(1)}, Q^{(2)}, \ldots, Q^{(L)}]
$$

Для каждого $Q^{(l)}$ предсказываем классы и bbox, и суммируем все losses:
$$
\mathcal{L} = \sum_{l=1}^{L} \mathcal{L}(Q^{(l)}, \text{targets})
$$

Финальный выход — это всё равно последний слой $Q^{(L)}$, но обучение идёт быстрее и стабильнее.


## Step 6: Deep Supervision


In [8]:
class TransformerDecoderWithIntermediates(nn.TransformerDecoder):
    def forward(self, tgt, memory, return_intermediate=False):
        output = tgt
        intermediates = []
        
        for mod in self.layers:
            output = mod(output, memory)
            if return_intermediate:
                intermediates.append(output)
        
        if self.norm is not None:
            output = self.norm(output)
        
        return torch.stack(intermediates) if return_intermediate else output

class Step6(Step5):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100, nhead=8, enc_layers=6, dec_layers=6):
        super().__init__(num_classes, emb_dim, num_queries, nhead, enc_layers, dec_layers)
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=emb_dim,
            nhead=nhead,
            dim_feedforward=emb_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_decoder = TransformerDecoderWithIntermediates(decoder_layer, num_layers=dec_layers)
    
    def forward(self, x):
        B = x.shape[0]
        features = Step1.forward(self, x)
        pos_emb = self.pos_emb(features)
        
        mem = features.flatten(2).permute(0, 2, 1)
        mem_pos = pos_emb.flatten(2).permute(0, 2, 1)
        
        mem = self.transformer_encoder(mem + mem_pos)
        
        queries = self.queries.weight.unsqueeze(0).expand(B, -1, -1)
        q_pos = self.query_pos.weight.unsqueeze(0).expand(B, -1, -1)
        
        intermediate_queries = self.transformer_decoder(queries + q_pos, mem + mem_pos, return_intermediate=True)
        
        outputs = []
        for layer_queries in intermediate_queries:
            class_logits = self.class_head(layer_queries)
            bbox_pred = self.bbox_head(layer_queries).sigmoid()
            outputs.append({'class_logits': class_logits, 'bbox_pred': bbox_pred})
        
        final = outputs[-1]
        final['aux_outputs'] = outputs[:-1]
        return final

model = Step6(num_classes=80, enc_layers=6, dec_layers=6)
out = model(torch.randn(2, 3, 640, 640))
print(f"Step 6: {len(out['aux_outputs'])} aux outputs")
print(f"Output: class_logits={out['class_logits'].shape}, bbox_pred={out['bbox_pred'].shape}")


Step 6: 5 aux outputs
Output: class_logits=torch.Size([2, 100, 81]), bbox_pred=torch.Size([2, 100, 4])


### Hungarian Matching: как сопоставить предсказания с ground truth?

У нас есть $N=100$ queries, но в изображении только несколько объектов. Какой query должен предсказывать какой объект? Это проблема bipartite matching.

Решение: Hungarian algorithm. Идея:
1. Строим cost matrix $C \in \mathbb{R}^{N \times M}$, где $M$ — число GT объектов
2. $C_{ij}$ — стоимость назначения query $i$ на GT объект $j$
3. Ищем оптимальное назначение, минимизирующее суммарную стоимость

Cost состоит из трёх компонентов:
$$
C_{ij} = \lambda_{\text{cls}} \cdot \mathcal{L}_{\text{cls}}(p_i, y_j) + \lambda_{\text{bbox}} \cdot \mathcal{L}_{\text{L1}}(b_i, \hat{b}_j) + \lambda_{\text{giou}} \cdot \mathcal{L}_{\text{GIoU}}(b_i, \hat{b}_j)
$$

где:
- $\mathcal{L}_{\text{cls}}$ — classification cost (negative probability правильного класса)
- $\mathcal{L}_{\text{L1}}$ — L1 distance между предсказанным bbox и GT bbox
- $\mathcal{L}_{\text{GIoU}}$ — Generalized IoU loss

После matching знаем, какие queries matched, остальные должны предсказывать "no object".


## Step 7: Add Hungarian Matcher


In [9]:
from scipy.optimize import linear_sum_assignment

def box_cxcywh_to_xyxy(boxes):
    cx, cy, w, h = boxes.unbind(-1)
    x1 = cx - 0.5 * w
    y1 = cy - 0.5 * h
    x2 = cx + 0.5 * w
    y2 = cy + 0.5 * h
    return torch.stack([x1, y1, x2, y2], dim=-1)

def box_iou(boxes1, boxes2):
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])
    
    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])
    
    wh = (rb - lt).clamp(min=0)
    inter = wh[:, :, 0] * wh[:, :, 1]
    
    union = area1[:, None] + area2 - inter
    iou = inter / union
    return iou

def generalized_box_iou(boxes1, boxes2):
    iou = box_iou(boxes1, boxes2)
    
    lt = torch.min(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.max(boxes1[:, None, 2:], boxes2[:, 2:])
    
    wh = (rb - lt).clamp(min=0)
    area = wh[:, :, 0] * wh[:, :, 1]
    
    return iou - (area - (box_iou(boxes1, boxes2) * area)) / area

class Step7(Step6):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100, nhead=8, enc_layers=6, dec_layers=6):
        super().__init__(num_classes, emb_dim, num_queries, nhead, enc_layers, dec_layers)
        self.cost_class = 2.0
        self.cost_bbox = 5.0
        self.cost_giou = 2.0
    
    @torch.no_grad()
    def match(self, outputs, targets):
        B, Q = outputs['class_logits'].shape[:2]
        class_probs = F.softmax(outputs['class_logits'], dim=-1)
        bbox_pred = outputs['bbox_pred']
        
        indices = []
        for b in range(B):
            if len(targets[b]['labels']) == 0:
                indices.append((torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)))
                continue
            
            gt_labels = targets[b]['labels']
            gt_boxes = targets[b]['boxes']
            
            cost_class = -class_probs[b][:, gt_labels]
            cost_bbox = torch.cdist(bbox_pred[b], gt_boxes, p=1)
            
            pred_boxes_xyxy = box_cxcywh_to_xyxy(bbox_pred[b])
            gt_boxes_xyxy = box_cxcywh_to_xyxy(gt_boxes)
            cost_giou = -generalized_box_iou(pred_boxes_xyxy, gt_boxes_xyxy)
            
            cost_matrix = self.cost_class * cost_class + self.cost_bbox * cost_bbox + self.cost_giou * cost_giou
            pred_idx, gt_idx = linear_sum_assignment(cost_matrix.cpu().numpy())
            indices.append((torch.tensor(pred_idx, dtype=torch.long), torch.tensor(gt_idx, dtype=torch.long)))
        
        return indices

model = Step7(num_classes=80, enc_layers=6, dec_layers=6)
outputs = model(torch.randn(2, 3, 640, 640))
targets = [
    {'labels': torch.tensor([1, 2, 5]), 'boxes': torch.rand(3, 4)},
    {'labels': torch.tensor([0, 3]), 'boxes': torch.rand(2, 4)}
]
matches = model.match(outputs, targets)
print(f"Step 7: Matched {[len(m[0]) for m in matches]} queries per image")


/home/tam2511/venvs/train_py10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tam2511/venvs/train_py10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Step 7: Matched [3, 2] queries per image


### DETR Loss: комбинация трёх компонентов

После matching можем посчитать loss. Используем три компонента:

**1. Classification Loss** — cross-entropy для всех $N$ queries:
$$
\mathcal{L}_{\text{cls}} = -\frac{1}{N} \sum_{i=1}^{N} \log p_i(y_i)
$$
где $y_i$ — matched класс (или "no object" для unmatched queries).

**2. L1 Loss** — только для matched queries:
$$
\mathcal{L}_{\text{L1}} = \frac{1}{M} \sum_{i \in \text{matched}} \|b_i - \hat{b}_i\|_1
$$
L1 loss измеряет абсолютное отклонение координат bbox.

**3. GIoU Loss** — только для matched queries:
$$
\mathcal{L}_{\text{GIoU}} = \frac{1}{M} \sum_{i \in \text{matched}} (1 - \text{GIoU}(b_i, \hat{b}_i))
$$
GIoU (Generalized IoU) — это улучшенная версия IoU, которая учитывает не только пересечение, но и форму описывающего прямоугольника.

Итоговый loss:
$$
\mathcal{L} = \lambda_{\text{cls}} \mathcal{L}_{\text{cls}} + \lambda_{\text{L1}} \mathcal{L}_{\text{L1}} + \lambda_{\text{GIoU}} \mathcal{L}_{\text{GIoU}}
$$

С deep supervision суммируем losses со всех слоёв.


## Step 8: Add Loss Computation


In [10]:
def giou_loss(boxes1, boxes2):
    boxes1_xyxy = box_cxcywh_to_xyxy(boxes1)
    boxes2_xyxy = box_cxcywh_to_xyxy(boxes2)
    
    giou = torch.diag(generalized_box_iou(boxes1_xyxy, boxes2_xyxy))
    return (1 - giou).mean()

class Step8(Step7):
    def __init__(self, num_classes=80, emb_dim=256, num_queries=100, nhead=8, enc_layers=6, dec_layers=6, w_class=2.0, w_bbox=5.0, w_giou=2.0):
        super().__init__(num_classes, emb_dim, num_queries, nhead, enc_layers, dec_layers)
        self.w_class = w_class
        self.w_bbox = w_bbox
        self.w_giou = w_giou
    
    def compute_loss(self, outputs, targets):
        indices = self.match(outputs, targets)
        loss = self._compute_single_loss(outputs, targets, indices)
        
        if 'aux_outputs' in outputs:
            for aux_out in outputs['aux_outputs']:
                aux_indices = self.match(aux_out, targets)
                loss += self._compute_single_loss(aux_out, targets, aux_indices)
        
        return loss
    
    def _compute_single_loss(self, outputs, targets, indices):
        B, Q = outputs['class_logits'].shape[:2]
        target_classes = torch.full((B, Q), self.num_classes, dtype=torch.long, device=outputs['class_logits'].device)
        
        for b, (pred_idx, gt_idx) in enumerate(indices):
            if len(pred_idx) > 0:
                target_classes[b, pred_idx] = targets[b]['labels'][gt_idx]
        
        loss_class = F.cross_entropy(outputs['class_logits'].flatten(0, 1), target_classes.flatten())
        
        loss_bbox, loss_giou, num_boxes = 0, 0, 0
        for b, (pred_idx, gt_idx) in enumerate(indices):
            if len(pred_idx) == 0:
                continue
            pred_boxes = outputs['bbox_pred'][b, pred_idx]
            gt_boxes = targets[b]['boxes'][gt_idx]
            loss_bbox += F.l1_loss(pred_boxes, gt_boxes, reduction='sum')
            loss_giou += giou_loss(pred_boxes, gt_boxes) * len(pred_idx)
            num_boxes += len(pred_idx)
        
        if num_boxes > 0:
            loss_bbox /= num_boxes
            loss_giou /= num_boxes
        
        return self.w_class * loss_class + self.w_bbox * loss_bbox + self.w_giou * loss_giou

model = Step8(num_classes=80, enc_layers=6, dec_layers=6)
outputs = model(torch.randn(2, 3, 640, 640))
targets = [
    {'labels': torch.tensor([1, 2, 5]), 'boxes': torch.rand(3, 4)},
    {'labels': torch.tensor([0, 3]), 'boxes': torch.rand(2, 4)}
]
loss = model.compute_loss(outputs, targets)
print(f"Step 8: Loss = {loss.item():.4f}")


Step 8: Loss = 94.2215


### Done

Мы построили полноценную архитектуру DETR из классического ResNet + FPN backbone. Текущая реализация включает все ключевые компоненты оригинального DETR:

1. **Backbone** — ResNet + FPN для извлечения признаков
2. **Queries** — learnable embeddings для поиска объектов
3. **Positional Encoding** — 2D sinusoidal позиции для feature map
4. **Transformer Encoder** — self-attention для обогащения feature map глобальным контекстом
5. **Transformer Decoder** — cross-attention между queries и encoded features
6. **Detection Heads** — предсказание классов и bounding boxes для каждого query
7. **Deep Supervision** — auxiliary losses для всех decoder layers
8. **Hungarian Matching** — оптимальное сопоставление предсказаний с GT через bipartite matching
9. **Combined Loss** — classification + L1 + GIoU loss

Основные преимущества DETR перед классическими детекторами:
- Не нужны anchors и их настройка
- Не нужен NMS для фильтрации дубликатов
- End-to-end differentiable pipeline
- и тд

Недостатки:
- Медленная сходимость (требует много эпох обучения)
- Плохо работает с маленькими объектами
- и тд

Эти проблемы решаются в более продвинутых версиях: Deformable DETR, Conditional DETR, DINO.

В целом по сравнению с mask2former тут мы реализовали detr полноценно :)